In [1]:
%%capture
!pip3 install datasets
!pip3 install transformers
!pip3 install sentencepiece

In [ ]:
!wget https://huggingface.co/datasets/vasudevgupta/data/resolve/main/test.csv

In [ ]:
from transformers import pipeline, PegasusForConditionalGeneration, PegasusTokenizer, GPT2LMHeadModel, GPT2Tokenizer

# for running GPT2-large:
# revision = "4c2361ee5aee22d127590fed42d2e0b7da3ffb0d"
# model_id = "vasudevgupta/dl-hack-gpt2-large"
# tokenizer = GPT2Tokenizer.from_pretrained(model_id, revision=revision)
# model = GPT2LMHeadModel.from_pretrained(model_id, revision=revision).to("cuda")

# for using distil-gpt2:
model_id = "vasudevgupta/dl-hack-distilgpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_id)
model = GPT2LMHeadModel.from_pretrained(model_id).to("cuda")

# for using pegasus-large:
# model_id = "vasudevgupta/dl-hack-pegasus-large"
# tokenizer = PegasusTokenizer.from_pretrained(model_id)
# model = PegasusForConditionalGeneration.from_pretrained(model_id).to("cuda")

In [ ]:
input_ids = tokenizer("Incorporating BERT into parallel sequence decoding with adapters", return_tensors="pt")['input_ids']
outputs = model.generate(input_ids.to("cuda"), num_beams=8, max_length=512, no_repeat_ngram_size=2)
tokenizer.decode(outputs[0])

In [ ]:
from datasets import load_dataset
from tqdm import tqdm

data = load_dataset("csv", data_files="test.csv")["train"]
data

In [ ]:
abstracts = []
titles = []
for sample in tqdm(data, total=len(data)):
  input_ids = tokenizer(sample['title'], return_tensors="pt")['input_ids']
  outputs = model.generate(input_ids.to("cuda"), num_beams=8, max_length=512, no_repeat_ngram_size=2)
  abstract = tokenizer.decode(outputs[0])
  print(abstract)
  abstracts.append(abstract)
  titles.append(sample['title'])

In [ ]:
import pandas as pd
df = pd.DataFrame.from_dict({"title": titles, "abstract": abstracts})
df.to_csv(f"predictions-{model_id}.csv", index=False)